## 均值方差

In [ ]:
import torch
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import transforms
from path import Path
from PIL import Image

images=Path(r'JPEGImages')

class CatData(Dataset):
    def __init__(self,files,transformer):
        super().__init__()
        self.transformer=transformer
        self.files=files
          
    def __getitem__(self, index):
        # 读取图片
        image_path = self.files[index]
        image = Image.open(image_path)      
        features = self.transformer(image)
        return features, 0
    
    def __len__(self):
        return len(self.files)

trans = transforms.Compose([
    transforms.Resize((410, 410)),
    transforms.ToTensor()
])

train_dataset = CatData(images.files(), trans)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)


def get_mean_std(loader):
    # Var[x] = E[X**2]-E[X]**2
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    # print(num_batches)
    # print(channels_sum)
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std


mean, std = get_mean_std(train_loader)

print(mean)
print(std)

## 权重删除

In [5]:
import torch
from torchvision.models import resnet50

model = resnet50()
parameters = model.state_dict()
ckpt = torch.load(r"C:\Users\hong\.cache\torch\hub\checkpoints\resnet50-0676ba61.pth")

for k in list(ckpt.keys()):
    if k not in parameters or ckpt[k].shape != parameters[k].shape:
        print(f"Removing key {k} from pretrained checkpoint")
        del ckpt[k]

['conv1.weight', 'bn1.running_mean', 'bn1.running_var', 'bn1.weight', 'bn1.bias', 'layer1.0.conv1.weight', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.conv2.weight', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.conv3.weight', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.1.conv1.weight', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.conv2.weight', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.conv3.weight', 'layer1.1.bn3.running_mean', 'layer1.1.bn3.running_var', 'layer1.1.bn3.weight', 'layer1.1.bn3.

## 模型冻结

In [ ]:
def freeze_bn(self):
    for m in self.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

if freeze_train:
    for param in model.extractor.parameters():
        param.requires_grad = False

## 权重初始化

In [ ]:
def weights_init(net, init_type='normal', init_gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and classname.find('Conv') != -1:
            if init_type == 'normal':
                torch.nn.init.normal_(m.weight.data, 0.0, init_gain)
            elif init_type == 'xavier':
                torch.nn.init.xavier_normal_(m.weight.data, gain=init_gain)
            elif init_type == 'kaiming':
                torch.nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                torch.nn.init.orthogonal_(m.weight.data, gain=init_gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
        elif classname.find('BatchNorm2d') != -1:
            torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
            torch.nn.init.constant_(m.bias.data, 0.0)
    print('initialize network with %s type' % init_type)
    net.apply(init_func)

## 混合精度

In [ ]:
from torch.cuda.amp import autocast,GradScaler 
scaler = GradScaler()
with autocast():
    losses = self.forward(imgs, bboxes, labels, scale)

#----------------------#
#   反向传播
#----------------------#
scaler.scale(losses[-1]).backward()
scaler.step(self.optimizer)
scaler.update()

## 修改模型

In [ ]:
from torchvision import models
from torch import nn
model = models.resnet50(weights=True)
num_ftrs = model.fc.in_features

 
for param in model.parameters():
    param.requires_grad = False

## conv1,fc是model的模块,in_features,out_features是Linear的属性,in_channels,out_channels是Conv2d的属性
model.conv1=  nn.Conv2d(1, model.conv1.out_channels, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(num_ftrs,2)